In [3]:
using Pkg
Pkg.add("JuMP")
Pkg.add("HiGHS")


   Resolving package versions...


  No Changes to `C:\Users\cooki\OneDrive\Documents\Ponts\2A\Rechop\Code\Project.toml`
  No Changes to `C:\Users\cooki\OneDrive\Documents\Ponts\2A\Rechop\Code\Manifest.toml`


   Resolving package versions...


  No Changes to `C:\Users\cooki\OneDrive\Documents\Ponts\2A\Rechop\Code\Project.toml`
  No Changes to `C:\Users\cooki\OneDrive\Documents\Ponts\2A\Rechop\Code\Manifest.toml`


In [25]:
using JuMP
using HiGHS

In [49]:
function variables_binaires!(model, tableau)
    for i in eachindex(tableau)
        @variable(model, tableau[i], Bin)
    end
end

function contrainteQ1!(model,mu,alpha,beta,M) # Quand on met la contrainte sur mu, alpha, beta existent deja, donc pas besoin de les def, @variable(model,mu)
    @constraint(model, mu >= 0)
    @constraint(model, mu <= M * alpha)
    @constraint(model, mu <= beta)
    @constraint(model, mu >= beta - (1 - alpha) * M)
end

function contrainteQ2!(model,mu,beta,M,y) #Attention, dire avant que y est binaire avec @variable(model, y, Bin) @variable(model,mu)
    @constraint(model, -beta <= M * y)
    @constraint(model, beta <= M * (1 - y))
    @constraint(model, mu >= 0)
    @constraint(model, mu >= beta)
    @constraint(model, mu <= M * (1 - y))
    @constraint(model, mu <= beta + M * y)   
end

function contrainteQ3!(model, mu, alpha, beta,M,x,y) # @variable(model, y, Bin)  @variable(model, x) @variable(model, mu)
    contrainteQ2!(model,x,beta-alpha,2*M,y)
    @constraint(model, mu == beta - x)
end


function contrainteQ4!(model,x,y,z,A,B,y) #déclarer @variable(model,y) 
    for i in 1:size(A, 1)
        @constraint(model, sum(A[i, j] * [x, y, z][j] for j in 1:3) <= B[i])
        
end

contrainteQ3! (generic function with 1 method)

In [53]:
M=10
alpha = 8
beta = 9
modeltest =  Model(HiGHS.Optimizer)
@variable(modeltest,y,Bin)
@variable(modeltest, x) 
@variable(modeltest,mu)
contrainteQ3!(modeltest,mu,alpha,beta,M,x,y)
@objective(modeltest,Min,mu)
optimize!(modeltest)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve: Optimal

Solving report
  Status            Optimal
  Primal bound      8
  Dual bound        8
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    8 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.00 (total)
                    0.00 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)
